In [1]:
import numpy as np
from scipy.optimize import fsolve, root
import warnings

In [2]:
def calculate_protein_level_with_regulation(params, hill_response=0.0, r_add=10, 
                                          splicing_half_life=7/60):
    """
    Calculate steady-state protein level given gene parameters and regulatory input.
    
    Args:
        params (array-like): Gene parameters in order:
            [k_on, k_off, burst_size, mrna_half_life, protein_half_life, protein_production_rate]
        hill_response (float): Desired Hill function output (0-1)
            0 = no regulation, 1 = maximum activation
        r_add (float): Maximum regulatory effect strength (default 10)
        splicing_half_life (float): mRNA maturation time (hours, default 7 min)
    
    Returns:
        float: Steady-state protein level
    """
    
    # Unpack parameters in order
    k_on, k_off, burst_size, mrna_half_life, protein_half_life, protein_production_rate = params
    
    # Convert half-lives to degradation rates
    mrna_deg_rate = np.log(2) / mrna_half_life
    protein_deg_rate = np.log(2) / protein_half_life
    splicing_rate = np.log(2) / splicing_half_life
    
    # Calculate transcription rate from burst parameters
    transcription_rate = burst_size * k_off
    
    # Apply regulation: k_on_eff = k_on + r_add * hill_response
    k_on_eff = k_on + r_add * hill_response
    k_on_eff = max(k_on_eff, 1e-10)  # Ensure positive
    
    # Calculate steady-state levels through the cascade
    # 1. Bursting probability
    burst_prob = k_on_eff / (k_on_eff + k_off)
    
    # 2. Unspliced mRNA level
    unspliced_mrna = transcription_rate * burst_prob / (mrna_deg_rate + splicing_rate)
    
    # 3. Spliced mRNA level  
    spliced_mrna = unspliced_mrna * splicing_rate / mrna_deg_rate
    
    # 4. Protein level
    protein_level = spliced_mrna * protein_production_rate / protein_deg_rate
    
    return protein_level

In [9]:
# params = [0.4,69.58,29.9,4.96,17.514,0.0297]
params = [0.212,31.93,79.28,5.54,88.79,0.14]
protein_level_no_reg = calculate_protein_level_with_regulation(params)
protein_level_half_maximal = calculate_protein_level_with_regulation(params, r_add = 1, hill_response = 0.5)
protein_level_maximal = calculate_protein_level_with_regulation(params, r_add = 1, hill_response = 1)
print(protein_level_no_reg, protein_level_half_maximal, protein_level_maximal)

2343.827446672802 7751.145773988586 12995.308035668148


In [2]:
def get_coupled_steady_state(
    # TF gene parameters
    k_on_TF,
    k_off_TF,
    basal_TF_rate,
    TF_mRNA_degradation_rate,
    splicing_rate_TF,
    TF_protein_production_rate,
    TF_protein_degradation_rate,
    
    # Target gene parameters
    k_on_Target,
    k_off_Target,
    basal_Target_rate,
    Target_mRNA_degradation_rate,
    splicing_rate_Target,
    Target_protein_production_rate,
    Target_protein_degradation_rate,
    max_effect=16.0,       # Maximum effect of regulation (can be adjusted)
    # Hill function parameters - can be fixed values or 'auto' for self-consistent
    K_TF='auto',          # Half-saturation for Target protein regulating TF
    K_Target='auto',      # Half-saturation for TF protein regulating Target
    n_TF=2,               # Hill coefficient for Target → TF regulation
    n_Target=2,           # Hill coefficient for TF → Target regulation
    
    # Solver options
    initial_guess=None,
    method='hybr',
    find_all_solutions=False,
    max_attempts=100
):
    """
    Find steady-state protein and mRNA levels for two mutually regulating genes.
    
    Parameters:
    -----------
    Gene-specific parameters for TF and Target genes including:
    - Transcriptional bursting (k_on, k_off)
    - Transcription rates (basal, max)
    - mRNA processing (degradation, splicing)
    - Protein production and degradation
    
    Hill function parameters:
    - K_TF, K_Target: half-saturation constants ('auto' sets K = steady-state protein level)
    - n_TF, n_Target: Hill coefficients (cooperativity)
    
    Returns:
    --------
    dict: Steady-state values for proteins and mRNAs, plus solver info
    """
    
    def steady_state_system(vars):
        """Define the coupled steady-state equations with self-consistent K values"""
        TF_protein, Target_protein = vars
        
        # Ensure non-negative protein levels
        TF_protein = max(TF_protein, 1e-12)
        Target_protein = max(Target_protein, 1e-12)
        
        # Set K values - either fixed or self-consistent
        K_TF_val = TF_protein if K_TF == 'auto' else K_TF
        K_Target_val = Target_protein if K_Target == 'auto' else K_Target
        
        # Hill functions for regulation
        # Target protein regulates TF gene (K_TF is the TF protein steady state)
        Target_hill = (Target_protein**n_TF) / (K_TF_val**n_TF + Target_protein**n_TF)
        
        # TF protein regulates Target gene (K_Target is the Target protein steady state)
        TF_hill = (TF_protein**n_Target) / (K_Target_val**n_Target + TF_protein**n_Target)
        
        # Effective k_on rates modulated by Hill functions
        k_on_TF_effective = k_on_TF #* max_effect * Target_hill
        k_on_Target_effective = k_on_Target * max_effect * TF_hill
        
        # TF gene expression pathway with effective k_on
        TF_bursting_fraction = k_on_TF_effective / (k_on_TF_effective + k_off_TF)
        TF_transcription_rate = basal_TF_rate
        TF_unspliced_mRNA = (TF_transcription_rate * TF_bursting_fraction / 
                            (TF_mRNA_degradation_rate + splicing_rate_TF))
        TF_spliced_mRNA = (TF_unspliced_mRNA * splicing_rate_TF / 
                          TF_mRNA_degradation_rate)
        TF_protein_predicted = (TF_spliced_mRNA * TF_protein_production_rate / 
                               TF_protein_degradation_rate)
        
        # Target gene expression pathway with effective k_on
        Target_bursting_fraction = k_on_Target_effective / (k_on_Target_effective + k_off_Target)
        Target_transcription_rate = basal_Target_rate
        Target_unspliced_mRNA = (Target_transcription_rate * Target_bursting_fraction / 
                                (Target_mRNA_degradation_rate + splicing_rate_Target))
        Target_spliced_mRNA = (Target_unspliced_mRNA * splicing_rate_Target / 
                              Target_mRNA_degradation_rate)
        Target_protein_predicted = (Target_spliced_mRNA * Target_protein_production_rate / 
                                   Target_protein_degradation_rate)
        
        # Return residuals (should be zero at steady state)
        residual_TF = TF_protein - TF_protein_predicted
        residual_Target = Target_protein - Target_protein_predicted
        
        return [residual_TF, residual_Target]
    
    def calculate_steady_state_outputs(TF_protein_ss, Target_protein_ss):
        """Calculate all steady-state quantities given protein levels"""
        # Set K values for output calculation
        K_TF_val = np.sqrt(max_effect-1)*TF_protein_ss if K_TF == 'auto' else K_TF
        K_Target_val = np.sqrt(max_effect-1)*Target_protein_ss if K_Target == 'auto' else K_Target
        
        # Recalculate Hill functions
        Target_hill = (Target_protein_ss**n_TF) / (K_TF_val**n_TF + Target_protein_ss**n_TF)
        TF_hill = (TF_protein_ss**n_Target) / (K_Target_val**n_Target + TF_protein_ss**n_Target)
        
        # Calculate transcription rates
        TF_transcription_rate = basal_TF_rate 
        Target_transcription_rate = basal_Target_rate
        
        # Calculate effective k_on rates
        k_on_TF_effective = k_on_TF #* max_effect * Target_hill
        k_on_Target_effective = k_on_Target * max_effect * TF_hill
        
        # Calculate all intermediate quantities
        # TF pathway
        TF_bursting_fraction = k_on_TF_effective / (k_on_TF_effective + k_off_TF)
        TF_unspliced_mRNA = (TF_transcription_rate * TF_bursting_fraction / 
                            (TF_mRNA_degradation_rate + splicing_rate_TF))
        TF_spliced_mRNA = (TF_unspliced_mRNA * splicing_rate_TF / 
                          TF_mRNA_degradation_rate)
        
        # Target pathway
        Target_bursting_fraction = k_on_Target_effective / (k_on_Target_effective + k_off_Target)
        Target_unspliced_mRNA = (Target_transcription_rate * Target_bursting_fraction / 
                                (Target_mRNA_degradation_rate + splicing_rate_Target))
        Target_spliced_mRNA = (Target_unspliced_mRNA * splicing_rate_Target / 
                              Target_mRNA_degradation_rate)
        
        return {
            'TF_protein': TF_protein_ss,
            'TF_spliced_mRNA': TF_spliced_mRNA,
            'TF_unspliced_mRNA': TF_unspliced_mRNA,
            'TF_transcription_rate': TF_transcription_rate,
            'TF_bursting_fraction': TF_bursting_fraction,
            'Target_protein': Target_protein_ss,
            'Target_spliced_mRNA': Target_spliced_mRNA,
            'Target_unspliced_mRNA': Target_unspliced_mRNA,
            'Target_transcription_rate': Target_transcription_rate,
            'Target_bursting_fraction': Target_bursting_fraction,
            'Target_hill_function': TF_hill,
            'TF_hill_function': Target_hill,
            'K_TF_used': K_TF_val,
            'K_Target_used': K_Target_val,
            'k_on_TF_effective': k_on_TF_effective,
            'k_on_Target_effective': k_on_Target_effective
        }
    
    # Set up initial guesses
    if initial_guess is None:
        # Default: try multiple initial guesses
        initial_guesses = [
            [10.0, 10.0],           # Both moderate
            [0.1, 0.1],           # Both low
            [100.0, 100.0],         # Both high
            [0.1, 100.0],          # TF low, Target high
            [100.0, 0.1],          # TF high, Target low
        ]
    else:
        initial_guesses = [initial_guess]
    
    solutions = []
    solver_info = []
    
    # Try to find solutions with different initial guesses
    for i, guess in enumerate(initial_guesses):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                
                if method == 'fsolve':
                    sol = fsolve(steady_state_system, guess, full_output=True)
                    solution, info, ier, msg = sol
                    success = (ier == 1)
                else:
                    sol = root(steady_state_system, guess, method=method)
                    solution = sol.x
                    success = sol.success
                    msg = sol.message if hasattr(sol, 'message') else 'No message'
                
                # Check if solution is valid (positive proteins, small residual)
                if success and all(x > 0 for x in solution):
                    residual = steady_state_system(solution)
                    residual_norm = np.linalg.norm(residual)
                    
                    if residual_norm < 1e-6:  # Solution tolerance
                        # Check if this solution is already found
                        is_new = True
                        for existing_sol, _ in solutions:
                            if np.allclose(solution, existing_sol, rtol=1e-4):
                                is_new = False
                                break
                        
                        if is_new:
                            solutions.append((solution, residual_norm))
                            solver_info.append({
                                'initial_guess': guess,
                                'residual_norm': residual_norm,
                                'message': msg,
                                'method': method
                            })
                            
                            if not find_all_solutions:
                                break
        
        except Exception as e:
            continue
    
    if not solutions:
        raise ValueError(f"No steady-state solution found after {len(initial_guesses)} attempts. "
                        "Try adjusting parameters or initial guesses.")
    
    # Return the best solution (lowest residual)
    best_solution, best_residual = min(solutions, key=lambda x: x[1])
    TF_protein_ss, Target_protein_ss = best_solution
    
    # Calculate all steady-state quantities
    results = calculate_steady_state_outputs(TF_protein_ss, Target_protein_ss)
    
    # Add solver information
    results['solver_info'] = {
        'num_solutions_found': len(solutions),
        'best_residual': best_residual,
        'all_solutions': solutions if find_all_solutions else [best_solution],
        'solver_details': solver_info
    }
    
    return results

# Example usage and test function
def example_mutual_activation():
    """Example: Two genes that activate each other with self-consistent K values"""
    
    params = {
        # TF gene parameters
        'k_on_TF': 0.25,
        'k_off_TF': 8.4,
        'basal_TF_rate': 32 * 8.4,
        'TF_mRNA_degradation_rate': np.log(2)/2.5,
        'splicing_rate_TF': np.log(2) / (7 / 60),
        'TF_protein_production_rate': 0.059,
        'TF_protein_degradation_rate': np.log(2) / 28, 
        
        # Target gene parameters
        'k_on_Target': 0.25,
        'k_off_Target': 8.4,
        'basal_Target_rate': 32 * 8.4,
        'Target_mRNA_degradation_rate': np.log(2)/2.5,
        'splicing_rate_Target':  np.log(2) / (7 / 60),
        'Target_protein_production_rate': 0.059,
        'Target_protein_degradation_rate': np.log(2) / 28,
        
        # Hill function parameters - using self-consistent K values
        'K_TF': 'auto',        # K will equal TF steady-state protein level
        'K_Target': 'auto',    # K will equal Target steady-state protein level
        'n_TF': 2,
        'n_Target': 2,
        
        # Solver options
        'find_all_solutions': True
    }
    
    result = get_coupled_steady_state(**params)
    
    print("Steady-state solution with self-consistent K values and effective k_on:")
    print(f"TF protein: {result['TF_protein']:.3f}")
    print(f"Target protein: {result['Target_protein']:.3f}")
    print(f"K_TF used: {result['K_TF_used']:.3f} (= TF protein level)")
    print(f"K_Target used: {result['K_Target_used']:.3f} (= Target protein level)")
    print(f"k_on_TF_effective: {result['k_on_TF_effective']:.4f} (vs base {params['k_on_TF']:.3f})")
    print(f"k_on_Target_effective: {result['k_on_Target_effective']:.4f} (vs base {params['k_on_Target']:.3f})")
    print(f"TF mRNA: {result['TF_spliced_mRNA']:.3f}")
    print(f"Target mRNA: {result['Target_spliced_mRNA']:.3f}")
    print(f"TF Hill function value: {result['TF_hill_function']:.3f}")
    print(f"Target Hill function value: {result['Target_hill_function']:.3f}")
    print(f"Solutions found: {result['solver_info']['num_solutions_found']}")
    print(f"Residual: {result['solver_info']['best_residual']:.2e}")
    
    return result

if __name__ == "__main__":
    example_mutual_activation()

Steady-state solution with self-consistent K values and effective k_on:
TF protein: 63.803
Target protein: 150.041
K_TF used: 247.110 (= TF protein level)
K_Target used: 581.108 (= Target protein level)
k_on_TF_effective: 0.2500 (vs base 0.250)
k_on_Target_effective: 0.0476 (vs base 0.250)
TF mRNA: 26.771
Target mRNA: 5.224
TF Hill function value: 0.269
Target Hill function value: 0.012
Solutions found: 1
Residual: 2.84e-14


In [6]:
import numpy as np
from scipy.integrate import solve_ivp

def get_steady_state_proteins(
    k_on_TF, k_off_TF, TF_transcription_rate, TF_mRNA_degradation_rate, splicing_rate, TF_protein_production_rate, TF_protein_degradation_rate,
    k_on_Target, k_off_Target, Target_transcription_rate, Target_mRNA_degradation_rate, Target_protein_production_rate, Target_protein_degradation_rate,
    max_effect=16.0, TF_hill=0.5, t_max=1000
):
    """Returns TF and Target steady-state protein levels."""
    
    # def odes(t, y):
    #     TF_u, TF_s, TF_p, T_u, T_s, T_p = y
        
    #     # Effective rates
    #     k_TF_eff = k_on_TF
    #     k_T_eff = k_on_Target * max_effect * TF_hill
        
    #     # Burst fractions
    #     b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
    #     b_T = k_T_eff / (k_T_eff + k_off_Target)
        
    #     return [
    #         basal_TF_rate * b_TF - TF_u * (TF_mRNA_deg + splicing_TF),
    #         TF_u * splicing_TF - TF_s * TF_mRNA_deg,
    #         TF_s * TF_prod - TF_p * TF_deg,
    #         basal_Target_rate * b_T - T_u * (Target_mRNA_deg + splicing_Target),
    #         T_u * splicing_Target - T_s * Target_mRNA_deg,
    #         T_s * Target_prod - T_p * Target_deg
    #     ]
    def odes(t, y):
        TF_u, TF_s, TF_p, T_u, T_s, T_p = y
        
        # Effective rates (symmetric regulation)
        k_TF_eff = k_on_TF * max_effect * TF_hill  # Target regulates TF
        k_T_eff = k_on_Target * max_effect * TF_hill  # TF regulates Target
        
        # Burst fractions
        b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
        b_T = k_T_eff / (k_T_eff + k_off_Target)
        
        return [
            TF_transcription_rate * b_TF - TF_u * (TF_mRNA_degradation_rate + splicing_rate),
            TF_u * splicing_rate - TF_s * TF_mRNA_degradation_rate,
            TF_s * TF_protein_production_rate - TF_p * TF_protein_degradation_rate,
            Target_transcription_rate * b_T - T_u * (Target_mRNA_degradation_rate + splicing_rate),
            T_u * splicing_rate - T_s * Target_mRNA_degradation_rate,
            T_s * Target_protein_production_rate - T_p * Target_protein_degradation_rate
        ]
    
    sol = solve_ivp(odes, [0, t_max], [1, 1, 1, 1, 1, 1], method='RK45', rtol=1e-8)
    return sol.y[2, -1], sol.y[5, -1]  # TF_protein, Target_protein

# Example
TF_ss, Target_ss = get_steady_state_proteins(
    0.25, 8.4, 268.8, np.log(2)/2.5, np.log(2)/(7/60), 0.059, np.log(2)/28,
    0.25, 8.4, 268.8, np.log(2)/2.5, 0.059, np.log(2)/28
)
print(f"TF: {TF_ss:.3f}, Target: {Target_ss:.3f}")

TF: 424.538, Target: 424.538


In [3]:
63.803/424.538

0.15028807786346568

In [8]:
# TF gene parameters
param_dict = {
'k_on_TF': 0.25,
'k_off_TF': 8.4,
'TF_transcription_rate': 32 * 8.4,
'TF_mRNA_degradation_rate': np.log(2)/2.5,
'splicing_rate_TF': np.log(2) / (7 / 60),
'TF_protein_production_rate': 0.059,
'TF_protein_degradation_rate': np.log(2) / 28, 

# Target gene parameters
'k_on_Target': 0.25,
'k_off_Target': 8.4,
'Target_transcription_rate': 32 * 8.4,
'Target_mRNA_degradation_rate': np.log(2)/2.5,
'splicing_rate':  np.log(2) / (7 / 60),
'Target_protein_production_rate': 0.059,
'Target_protein_degradation_rate': np.log(2) / 28,

# Hill function parameters - using self-consistent K values
'K_TF': 'auto',        # K will equal TF steady-state protein level
'K_Target': 'auto',    # K will equal Target steady-state protein level
'n_TF': 2,
'n_Target': 2}

In [8]:
mRNA_level = (8*param_dict['k_on_TF'] /(8*param_dict['k_on_TF'] + param_dict['k_off_TF']))*(param_dict['basal_TF_rate']/(param_dict['TF_mRNA_degradation_rate'] + param_dict['splicing_rate_TF']))
spliced_mRNA_level = mRNA_level * (param_dict['splicing_rate_TF'] / param_dict['TF_mRNA_degradation_rate'])
protein_level = spliced_mRNA_level * (param_dict['TF_protein_production_rate'] / param_dict['TF_protein_degradation_rate'])
print(f"TF mRNA level: {mRNA_level:.3f}, Spliced mRNA level: {spliced_mRNA_level:.3f}, Protein level: {protein_level:.3f}")

TF mRNA level: 8.313, Spliced mRNA level: 178.128, Protein level: 424.538


In [15]:
#to get TF protein mean for TF protein->Target k_on link function
def get_averages(k_on_TF=None,
                 k_off_TF=None,
                 TF_transcription_rate=None,
                 TF_mRNA_degradation_rate=None,
                 splicing_rate=None,
                 protein_production_rate=None,
                 protein_degradation_rate=None,
                 **other_kwargs):
    average_bursting = k_on_TF / (k_on_TF + k_off_TF)
    average_unspliced_mRNA = TF_transcription_rate * average_bursting / (TF_mRNA_degradation_rate + splicing_rate)
    average_spliced_mRNA = average_unspliced_mRNA * splicing_rate / TF_mRNA_degradation_rate
    average_protein = average_spliced_mRNA * protein_production_rate / protein_degradation_rate
    return dict(average_spliced_mRNA=average_spliced_mRNA, average_protein=average_protein)

k_on_TF = 0.25
k_off_TF = 8.4
TF_transcription_rate = 32 * k_off_TF
TF_mRNA_degradation_rate = np.log(2) / 2.5
splicing_rate = np.log(2) / (7 / 60)
protein_production_rate = 0.059
TF_protein_degradation_rate = np.log(2) / 28
a = get_averages(k_on_TF=k_on_TF,
                    k_off_TF=k_off_TF,
                    TF_transcription_rate=TF_transcription_rate,
                    TF_mRNA_degradation_rate=TF_mRNA_degradation_rate,
                    splicing_rate=splicing_rate,
                    protein_production_rate=protein_production_rate
                    , protein_degradation_rate=TF_protein_degradation_rate)
a.items()

dict_items([('average_spliced_mRNA', 26.770674887082127), ('average_protein', 63.80341167619444)])

In [16]:
kOnEff = k_on_TF
bursting_fraction = kOnEff / (kOnEff + k_off_TF)
mRNA_prod = TF_transcription_rate * bursting_fraction / (TF_mRNA_degradation_rate + splicing_rate)
splicesd_mRNA = mRNA_prod * splicing_rate / TF_mRNA_degradation_rate
protein_level = splicesd_mRNA * protein_production_rate / TF_protein_degradation_rate
print(splicesd_mRNA)

26.770674887082127


In [18]:
def get_steady_state_self_regulating_TF(
    k_on_TF, k_off_TF, TF_transcription_rate, TF_mRNA_degradation_rate, 
    splicing_rate, TF_protein_production_rate, TF_protein_degradation_rate,
    k_on_Target, k_off_Target, Target_transcription_rate, Target_mRNA_degradation_rate, 
    Target_protein_production_rate, Target_protein_degradation_rate,
    max_effect=16.0, TF_hill=0.5, **kwargs
):
    """
    Calculate steady state for system where:
    - TF regulates itself (self-regulation)
    - TF regulates Target 
    - Hill function = 0.5 at steady state (EC50 normalization)
    
    Returns steady-state protein levels for TF and Target
    """
    
    def steady_state_equations(TF_p_ss):
        """
        System of equations to solve for TF steady state.
        At steady state, Hill function = 0.5 by design.
        """
        
        # TF self-regulation: Hill function = 0.5 at steady state
        k_TF_eff = k_on_TF * max_effect * TF_hill  # TF_hill = 0.5
        b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
        
        # Calculate what TF protein level should be given this burst fraction
        TF_u_ss = (TF_transcription_rate * b_TF) / (TF_mRNA_degradation_rate + splicing_rate)
        TF_s_ss = (TF_u_ss * splicing_rate) / TF_mRNA_degradation_rate
        TF_p_predicted = (TF_s_ss * TF_protein_production_rate) / TF_protein_degradation_rate
        
        # This should equal our input TF_p_ss for self-consistency
        return TF_p_predicted - TF_p_ss
    
    # Solve for self-consistent TF protein level
    TF_p_ss = fsolve(steady_state_equations, 1.0)[0]
    
    # Now calculate Target steady state (regulated by TF)
    # Target is regulated by TF protein level
    k_Target_eff = k_on_Target * max_effect * TF_hill  # Same Hill value (0.5)
    b_Target = k_Target_eff / (k_Target_eff + k_off_Target)
    
    # Target steady states
    Target_u_ss = (Target_transcription_rate * b_Target) / (Target_mRNA_degradation_rate + splicing_rate)
    Target_s_ss = (Target_u_ss * splicing_rate) / Target_mRNA_degradation_rate
    Target_p_ss = (Target_s_ss * Target_protein_production_rate) / Target_protein_degradation_rate
    
    # Recalculate TF intermediates for completeness
    k_TF_eff = k_on_TF * max_effect * TF_hill
    b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
    TF_u_ss = (TF_transcription_rate * b_TF) / (TF_mRNA_degradation_rate + splicing_rate)
    TF_s_ss = (TF_u_ss * splicing_rate) / TF_mRNA_degradation_rate
    
    return {
        'average_protein_TF': TF_p_ss,
        'average_protein_Target': Target_p_ss,
        # Additional info
    }

# Example usage
average_values = get_steady_state_self_regulating_TF(**param_dict)

In [19]:
average_values

{'average_protein_TF': 424.5380853839091,
 'average_protein_Target': 424.5380853839091}

# Generalized case


In [18]:
import numpy as np
from scipy.optimize import fsolve

def getSteadyStateFromMatrix(
    interactionMatrix, 
    geneNames,
    geneParams, 
    networkParams,
    initialGuess=None,
    **kwargs
):
    """
    Calculate steady state for gene regulatory network defined by interaction matrix.
    
    Parameters:
    -----------
    interactionMatrix : numpy.array
        Standard matrix notation where:
        - Rows are regulators (genes doing the regulating)
        - Columns are targets (genes being regulated)
        - Values indicate interaction strength/type:
          - 0: no interaction
          - positive: activation  
          - negative: repression
        Example: [[1, 1], [-1, 0]] means:
          Gene 0 activates itself and Gene 1
          Gene 1 represses Gene 0
          
    geneNames : list
        List of gene names corresponding to matrix rows/columns
        
    geneParams : dict
        Gene-specific parameters for each gene:
        {
            'gene_name': {
                'kOnBasal': basal transcription rate,
                'kOff': transcription off rate,
                'transcriptionRate': max transcription rate,
                'mrnaDecayRate': mRNA degradation rate,
                'splicingRate': splicing rate,
                'proteinProductionRate': protein production rate,
                'proteinDecayRate': protein degradation rate
            }
        }
        
    networkParams : dict
        Network-wide parameters:
        {
            'maxEffect': maximum regulatory effect,
            'hillCoeff': Hill coefficient for regulatory functions,
            'targetHillValue': target Hill function value (e.g., 0.5)
        }
        
    initialGuess : array-like, optional
        Initial guess for protein concentrations
        
    Returns:
    --------
    dict: Steady-state concentrations for all genes
    """
    
    # Ensure we have a numpy array
    interactionMatrix = np.array(interactionMatrix)
    numGenes = len(geneNames)
    
    # Extract network parameters
    maxEffect = networkParams.get('maxEffect', 16.0)
    hillCoeff = networkParams.get('hillCoeff', 1.0)
    targetHillValue = networkParams.get('targetHillValue', 0.5)
    
    def calculateHillFunction(regulatorConc, interactionStrength, kValue):
        """Calculate Hill function for given regulator concentration and interaction"""
        if interactionStrength == 0:
            return 0
        
        # Hill function: (regulator/K)^n / (1 + (regulator/K)^n) for activation
        # For simplicity, using n=1 (Hill coefficient = 1)
        if interactionStrength > 0:  # Activation
            hillValue = regulatorConc / (regulatorConc + kValue)
            return maxEffect * hillValue
        else:  # Repression
            hillValue = regulatorConc / (regulatorConc + kValue)
            return -maxEffect * hillValue
    
    def steadyStateEquations(proteinConcentrations):
        """
        System of equations for steady state calculation.
        One equation per gene.
        """
        equations = []
        
        for targetIdx, targetGene in enumerate(geneNames):
            targetParams = geneParams[targetGene]
            
            # Calculate total regulatory effect on this target
            totalRegulatoryEffect = 0
            for regulatorIdx in range(numGenes):
                # Standard matrix notation: interactionMatrix[regulator_row, target_column]
                interactionStrength = interactionMatrix[regulatorIdx, targetIdx]
                if interactionStrength != 0:
                    regulatorConc = proteinConcentrations[regulatorIdx]
                    # Use the regulator concentration as K value (self-consistent approach)
                    kValue = regulatorConc  # This makes Hill function = 0.5 at steady state
                    hillFunctionValue = calculateHillFunction(regulatorConc, interactionStrength, kValue)
                    totalRegulatoryEffect += hillFunctionValue
            
            # Calculate effective transcription parameters
            kOnEffective = targetParams['kOnBasal'] * (1 + totalRegulatoryEffect)
            kOff = targetParams['kOff']
            burstFraction = kOnEffective / (kOnEffective + kOff)
            
            # Calculate steady-state concentrations through the cascade
            unsplicedMrnaConc = (targetParams['transcriptionRate'] * burstFraction) / \
                              (targetParams['mrnaDecayRate'] + targetParams['splicingRate'])
            
            splicedMrnaConc = (unsplicedMrnaConc * targetParams['splicingRate']) / \
                            targetParams['mrnaDecayRate']
            
            predictedProteinConc = (splicedMrnaConc * targetParams['proteinProductionRate']) / \
                                 targetParams['proteinDecayRate']
            
            # Equation: predicted concentration should equal actual concentration
            equations.append(predictedProteinConc - proteinConcentrations[targetIdx])
        
        return equations
    
    # Set initial guess
    if initialGuess is None:
        initialGuess = np.ones(numGenes)
    
    # Solve the system of equations
    solutionConcentrations = fsolve(steadyStateEquations, initialGuess)
    
    # Prepare results dictionary
    results = {}
    for i, geneName in enumerate(geneNames):
        results[f'averageProtein_{geneName}'] = solutionConcentrations[i]
    
    # Also calculate intermediate values for completeness
    intermediateResults = {}
    for targetIdx, targetGene in enumerate(geneNames):
        targetParams = geneParams[targetGene]
        
        # Recalculate regulatory effects
        totalRegulatoryEffect = 0
        for regulatorIdx in range(numGenes):
            # Standard matrix notation: interactionMatrix[regulator_row, target_column]
            interactionStrength = interactionMatrix[regulatorIdx, targetIdx]
            if interactionStrength != 0:
                regulatorConc = solutionConcentrations[regulatorIdx]
                kValue = regulatorConc  # Self-consistent K value
                hillValue = calculateHillFunction(regulatorConc, interactionStrength, kValue)
                totalRegulatoryEffect += hillValue
        
        # Calculate intermediate concentrations
        kOnEffective = targetParams['kOnBasal'] * (1 + totalRegulatoryEffect)
        burstFraction = kOnEffective / (kOnEffective + targetParams['kOff'])
        
        unsplicedMrna = (targetParams['transcriptionRate'] * burstFraction) / \
                       (targetParams['mrnaDecayRate'] + targetParams['splicingRate'])
        
        splicedMrna = (unsplicedMrna * targetParams['splicingRate']) / \
                     targetParams['mrnaDecayRate']
        
        intermediateResults[f'unsplicedMrna_{targetGene}'] = unsplicedMrna
        intermediateResults[f'splicedMrna_{targetGene}'] = splicedMrna
        intermediateResults[f'burstFraction_{targetGene}'] = burstFraction
        
        # Also calculate Hill function values for verification
        for regulatorIdx in range(numGenes):
            interactionStrength = interactionMatrix[regulatorIdx, targetIdx]
            if interactionStrength != 0:
                regulatorGene = geneNames[regulatorIdx]
                regulatorConc = solutionConcentrations[regulatorIdx]
                kValue = regulatorConc
                hillFuncValue = calculateHillFunction(regulatorConc, interactionStrength, kValue) / maxEffect
                intermediateResults[f'hillFunction_{regulatorGene}To{targetGene}'] = hillFuncValue
    
    results.update(intermediateResults)
    return results

def createExampleNetwork():
    """Create example network using standard matrix notation"""
    
    # Standard matrix notation: rows = regulators, columns = targets
    # [[1, 1], [-1, 0]] means:
    # TF activates TF and Target
    # Target represses TF
    interactionMatrix = np.array([
        [0.0, 1.0],   # TF (row 0) -> TF, Target  
        [0.0, 0.0]   # Target (row 1) -> TF (repression), Target (none)
    ])
    
    geneNames = ['TF', 'Target']
    
    # Gene parameters
    geneParams = {
        'TF': {
            'kOnBasal': 0.1,  # k_on_TF from your original
            'kOff': 0.5,      # k_off_TF
            'transcriptionRate': 2.0,   # TF_transcription_rate
            'mrnaDecayRate': 0.3,       # TF_mRNA_degradation_rate
            'splicingRate': 0.2,        # splicing_rate
            'proteinProductionRate': 1.0, # TF_protein_production_rate
            'proteinDecayRate': 0.1     # TF_protein_degradation_rate
        },
        'Target': {
            'kOnBasal': 0.05,  # k_on_Target
            'kOff': 0.3,       # k_off_Target
            'transcriptionRate': 1.5,   # Target_transcription_rate
            'mrnaDecayRate': 0.25,      # Target_mRNA_degradation_rate
            'splicingRate': 0.2,        # same splicing rate
            'proteinProductionRate': 0.8, # Target_protein_production_rate
            'proteinDecayRate': 0.08    # Target_protein_degradation_rate
        }
    }
    
    # Network parameters
    networkParams = {
        'maxEffect': 16.0,
        'hillCoeff': 1.0,
        'targetHillValue': 0.5
    }
    
    return interactionMatrix, geneNames, geneParams, networkParams

# Example usage
if __name__ == "__main__":
    interactionMatrix, geneNames, geneParams, networkParams = createExampleNetwork()
    
    results = getSteadyStateFromMatrix(
        interactionMatrix, 
        geneNames,
        geneParams, 
        networkParams
    )
    
    print("Steady-state results:")
    for key, value in results.items():
        print(f"{key}: {value:.4f}")

Steady-state results:
averageProtein_TF: 4.4444
averageProtein_Target: 16.0000
unsplicedMrna_TF: 0.6667
splicedMrna_TF: 0.4444
burstFraction_TF: 0.1667
unsplicedMrna_Target: 2.0000
splicedMrna_Target: 1.6000
burstFraction_Target: 0.6000
hillFunction_TFToTarget: 0.5000


In [ ]:
import numpy as np
from scipy.optimize import fsolve

def generateReactionSystem(interactionMatrix, geneNames, geneParams, networkParams):
    """
    Generate reaction system from interaction matrix and parameters.
    
    Parameters:
    -----------
    interactionMatrix : numpy.array
        Standard matrix notation (rows=regulators, columns=targets)
    geneNames : list
        List of gene names
    geneParams : dict
        Gene-specific parameters
    networkParams : dict
        Network-wide parameters
        
    Returns:
    --------
    dict: Complete reaction system ready for steady-state solving
    """
    
    # Ensure we have a numpy array
    interactionMatrix = np.array(interactionMatrix)
    numGenes = len(geneNames)
    
    # Extract network parameters
    maxEffect = networkParams.get('maxEffect', 16.0)
    hillCoeff = networkParams.get('hillCoeff', 1.0)
    targetHillValue = networkParams.get('targetHillValue', 0.5)
    
    def calculateHillFunction(regulatorConc, interactionStrength, kValue):
        """Calculate Hill function for given regulator concentration and interaction"""
        if interactionStrength == 0:
            return 0
        
        # Hill function: (regulator/K)^n / (1 + (regulator/K)^n)
        if interactionStrength > 0:  # Activation
            hillValue = regulatorConc / (regulatorConc + kValue)
            return maxEffect * hillValue
        else:  # Repression
            hillValue = regulatorConc / (regulatorConc + kValue)
            return -maxEffect * hillValue
    
    def calculateGeneExpression(targetGeneIdx, proteinConcentrations):
        """
        Calculate expression levels for a single target gene given protein concentrations.
        
        Returns:
        --------
        dict: Expression levels (unspliced mRNA, spliced mRNA, protein, etc.)
        """
        targetGene = geneNames[targetGeneIdx]
        targetParams = geneParams[targetGene]
        
        # Calculate total regulatory effect on this target
        totalRegulatoryEffect = 0
        regulatoryDetails = {}
        
        for regulatorIdx in range(numGenes):
            interactionStrength = interactionMatrix[regulatorIdx, targetGeneIdx]
            if interactionStrength != 0:
                regulatorConc = proteinConcentrations[regulatorIdx]
                regulatorGene = geneNames[regulatorIdx]
                
                # Use self-consistent K value approach
                kValue = regulatorConc
                hillFunctionValue = calculateHillFunction(regulatorConc, interactionStrength, kValue)
                totalRegulatoryEffect += hillFunctionValue
                
                # Store details for debugging/analysis
                regulatoryDetails[f'{regulatorGene}_to_{targetGene}'] = {
                    'interactionStrength': interactionStrength,
                    'regulatorConc': regulatorConc,
                    'kValue': kValue,
                    'hillFunctionValue': hillFunctionValue,
                    'normalizedHillValue': hillFunctionValue / maxEffect if maxEffect != 0 else 0
                }
        
        # Calculate effective transcription parameters
        kOnEffective = targetParams['kOnBasal'] * (1 + totalRegulatoryEffect)
        kOff = targetParams['kOff']
        burstFraction = kOnEffective / (kOnEffective + kOff)
        
        # Calculate concentrations through the expression cascade
        unsplicedMrnaConc = (targetParams['transcriptionRate'] * burstFraction) / \
                          (targetParams['mrnaDecayRate'] + targetParams['splicingRate'])
        
        splicedMrnaConc = (unsplicedMrnaConc * targetParams['splicingRate']) / \
                        targetParams['mrnaDecayRate']
        
        proteinConc = (splicedMrnaConc * targetParams['proteinProductionRate']) / \
                     targetParams['proteinDecayRate']
        
        return {
            'totalRegulatoryEffect': totalRegulatoryEffect,
            'kOnEffective': kOnEffective,
            'kOff': kOff,
            'burstFraction': burstFraction,
            'unsplicedMrna': unsplicedMrnaConc,
            'splicedMrna': splicedMrnaConc,
            'protein': proteinConc,
            'regulatoryDetails': regulatoryDetails
        }
    
    # Package everything into a reaction system
    reactionSystem = {
        'interactionMatrix': interactionMatrix,
        'geneNames': geneNames,
        'geneParams': geneParams,
        'networkParams': networkParams,
        'numGenes': numGenes,
        'maxEffect': maxEffect,
        'calculateGeneExpression': calculateGeneExpression
    }
    
    return reactionSystem

def solveReactionSystemSteadyState(reactionSystem, initialGuess=None, **kwargs):
    """
    Solve steady state for a reaction system generated by generateReactionSystem.
    
    Parameters:
    -----------
    reactionSystem : dict
        Reaction system from generateReactionSystem
    initialGuess : array-like, optional
        Initial guess for protein concentrations
        
    Returns:
    --------
    dict: Steady-state results with detailed breakdown
    """
    
    geneNames = reactionSystem['geneNames']
    numGenes = reactionSystem['numGenes']
    calculateGeneExpression = reactionSystem['calculateGeneExpression']
    
    def steadyStateEquations(proteinConcentrations):
        """
        System of equations for steady state calculation.
        One equation per gene.
        """
        equations = []
        
        for targetIdx in range(numGenes):
            # Calculate predicted protein concentration
            expressionResults = calculateGeneExpression(targetIdx, proteinConcentrations)
            predictedProteinConc = expressionResults['protein']
            
            # Equation: predicted concentration should equal actual concentration
            equations.append(predictedProteinConc - proteinConcentrations[targetIdx])
        
        return equations
    
    # Set initial guess
    if initialGuess is None:
        initialGuess = np.ones(numGenes)
    
    # Solve the system of equations
    solutionConcentrations = fsolve(steadyStateEquations, initialGuess)
    
    # Calculate detailed results for each gene at steady state
    detailedResults = {}
    for targetIdx, geneName in enumerate(geneNames):
        expressionResults = calculateGeneExpression(targetIdx, solutionConcentrations)
        
        # Store main results
        detailedResults[f'averageProtein_{geneName}'] = solutionConcentrations[targetIdx]
        detailedResults[f'unsplicedMrna_{geneName}'] = expressionResults['unsplicedMrna']
        detailedResults[f'splicedMrna_{geneName}'] = expressionResults['splicedMrna']
        detailedResults[f'burstFraction_{geneName}'] = expressionResults['burstFraction']
        detailedResults[f'kOnEffective_{geneName}'] = expressionResults['kOnEffective']
        detailedResults[f'totalRegulatoryEffect_{geneName}'] = expressionResults['totalRegulatoryEffect']
        
        # Store regulatory details
        for regKey, regDetails in expressionResults['regulatoryDetails'].items():
            detailedResults[f'hillFunction_{regKey}'] = regDetails['normalizedHillValue']
            detailedResults[f'regulatoryEffect_{regKey}'] = regDetails['hillFunctionValue']
    
    # Add solution info
    detailedResults['solutionConcentrations'] = solutionConcentrations
    detailedResults['convergenceCheck'] = np.max(np.abs(steadyStateEquations(solutionConcentrations)))
    
    return detailedResults

def getSteadyStateFromMatrix(interactionMatrix, geneNames, geneParams, networkParams, initialGuess=None, **kwargs):
    """
    Convenience function that combines reaction generation and steady-state solving.
    
    This maintains backward compatibility with the original interface.
    """
    reactionSystem = generateReactionSystem(interactionMatrix, geneNames, geneParams, networkParams)
    return solveReactionSystemSteadyState(reactionSystem, initialGuess, **kwargs)

def createExampleNetwork():
    """Create example network using standard matrix notation"""
    
    # Standard matrix notation: rows = regulators, columns = targets
    # [[1, 1], [-1, 0]] means:
    # TF activates TF and Target
    # Target represses TF
    interactionMatrix = np.array([
        [1.0, 1.0],   # TF (row 0) -> TF, Target  
        [-1.0, 0.0]   # Target (row 1) -> TF (repression), Target (none)
    ])
    
    geneNames = ['TF', 'Target']
    
    # Gene parameters
    geneParams = {
        'TF': {
            'kOnBasal': 0.1,
            'kOff': 0.5,
            'transcriptionRate': 2.0,
            'mrnaDecayRate': 0.3,
            'splicingRate': 0.2,
            'proteinProductionRate': 1.0,
            'proteinDecayRate': 0.1
        },
        'Target': {
            'kOnBasal': 0.05,
            'kOff': 0.3,
            'transcriptionRate': 1.5,
            'mrnaDecayRate': 0.25,
            'splicingRate': 0.2,
            'proteinProductionRate': 0.8,
            'proteinDecayRate': 0.08
        }
    }
    
    # Network parameters
    networkParams = {
        'maxEffect': 16.0,
        'hillCoeff': 1.0,
        'targetHillValue': 0.5
    }
    
    return interactionMatrix, geneNames, geneParams, networkParams

# Example usage
if __name__ == "__main__":
    # Method 1: Two-step approach (recommended for debugging)
    interactionMatrix, geneNames, geneParams, networkParams = createExampleNetwork()
    
    print("=== Two-step approach ===")
    reactionSystem = generateReactionSystem(interactionMatrix, geneNames, geneParams, networkParams)
    results = solveReactionSystemSteadyState(reactionSystem)
    
    print("Steady-state results:")
    for key, value in results.items():
        if isinstance(value, (int, float)):
            print(f"{key}: {value:.4f}")
    
    print("\n=== One-step approach (backward compatibility) ===")
    results2 = getSteadyStateFromMatrix(interactionMatrix, geneNames, geneParams, networkParams)
    
    print("Steady-state results:")
    for key, value in results2.items():
        if isinstance(value, (int, float)):
            print(f"{key}: {value:.4f}")

## Using ODE45 to get steady state more accurately (specifically for the regulated)
